### Centros poblados
Desde el 2017 el INEI pone a disposición del público la información del CPV 2017 a nivel de centros poblados y manzadas. Esta información puede ser consultada desde la web alterna de Redatam destinada este fin (https://censos2017.inei.gob.pe/bininei/RpWebStats.exe/CmdSet?BASE=CPV2017&ITEM=PROGRED&lang=esp).

De forma similar a como ocurre con la información a nivel de distrito y provincia, los datos son devueltos por el sistema en forma de tablas individuales por Ubigeo (revisar el tutorial sobre distritos acá: link). Así, a pesar del avance que representa el acceso a este nivel de desagregación para el análisis de distintos fenómenos sociales y demográficos, su uso está restringido a la capacidad que el usuario tenga para procesar la información. Asimismo, la información georreferenciada de ccpp y manzanas es distribuida a través de un WMS, formato de tipo imagen que impide la vinculación de la información con los datos censales de caracter vectorial.

A pesar de estas limitaciones, Redapy resulta en una herramienta potente para reducir la brecha en la capacidad de análisis de los datos. A continuación, se presentan dos posibles usos de ambas bases de datos de centros poblados y manzanas haciendo uso de redapyx e información de acceso público

### CCPP en Redatam 

Lo primero será comparar el número total de ccpp registrados en los resultados finales del CPV 2017 con el número de ccpp disponibles en la base de datos de Redatam (resultados de:
https://www.inei.gob.pe/media/MenuRecursivo/publicaciones_digitales/Est/Lib1541/index.htm). Para ello utilizamos una función complementaria de Redapy denominada "clean_directorio_ccpp" la cual se encuentra en el submódulo utiles. Esta función fue desarrollada para facilitar la descarga y limpieza de las tablas de centros poblados distribuídas en formato excel.

Para su funcionamiento, se proporciona la ruta y se especifica el valor _True_ en el parámetro "nacional". Esto permite que la función descargue todas las tablas a nivel nacional y procese sus resultados hasta obtener una matriz cuyo index sera el ubigeo del ccpp. Note que si nuestro interés fuera solicitar solo la información para un determinado departamento, solo tendríamos que asegurarnos de pasar la ruta correcta con el número del departamento al final (_dpto01_)

In [1]:
import sys
sys.path.append("C:\\Users\\Guillermo\\Desktop\\Python\\05. Lib gestión pública")

import redapyx
import pandas as pd
import numpy as np

In [55]:
ruta="https://www.inei.gob.pe/media/MenuRecursivo/publicaciones_digitales/Est/Lib1541/cuadros/dpto01.xlsx"
ccpp=redapyx.utiles.clean_directorio_ccpp(ruta, nacional=True)

print("Total de centros poblados 2017:",ccpp.count()[1])

## Eliminamos los ccpp que no tienen población 
ccpp_1=ccpp.query('pob_censada>1')
print("Total de centros poblados con al menos 1 persona:",ccpp_1.count()[0])
ccpp_1.head(7)

Total de centros poblados 2017: 94922
Total de centros poblados con al menos 1 persona: 71984


,codigo,ccpp_nombre,region_natural,altitud,pob_censada,hob_censados,muj_censados,viv_particulares,viv_ocupadas,viv_des
ubigeo,,,,,,,,,,
0101010001,0001,CHACHAPOYAS,Quechua,2483,32026,15131,16895,9304,8593,711
0101010002,0002,CACLIC,Yunga fluvial,1745,41,29,12,13,10,3
0101010003,0003,VITALIANO,Quechua,2452,5,4,1,3,3,0
0101010009,0009,EL CRUCE,Yunga fluvial,1789,10,4,6,4,4,0
0101010010,0010,ACHAMAQUI,Yunga fluvial,1718,62,28,34,6,6,0
0101010012,0012,EL MOLINO,Yunga fluvial,2143,11,7,4,3,3,0
0101010013,0013,EL ALFALFAR,Yunga fluvial,2218,5,3,2,8,8,0


El total de ccpp registrados durante el censo 2017 fue de **94,922**, de los cuales **77,916** cuentan con población de acuerdo con los resultados definitivos del censo 2017.

Ahora que ya conocemos el número final de ccpp con población, procedemos a realizar la consulta a Redatam para conocer el universo de ccpp de los que podemos solicitar información. Para ello

In [4]:
df1=redapyx.get(type='frequency', var1='vivienda.VAREA', area_break="centro poblado", output_info=False)
# print("Total de centros poblados con al menos 1 persona:",df1.count()[0])
df1.head()

Table was scraped successfully in: 0:00:41.667947
Table was cleaned successfully in: 0:01:04.530180


,resp,fre,ubigeo
8,Urbano censal,9379,0101010001
14,Urbano censal,77,0101020001
20,Urbano censal,101,0101030001
26,Urbano censal,188,0101040001
32,Urbano censal,31,0101050001


Una vez realizada la consulta comprobamos que Redatam solo proporciona información de 3,718 ccpp de los 77,916, lo que representa solo el 6% del total de ccpp con población. Si revisamos estos datos con un corte de 50 personas (de acuerdo con el SICCEP), el total de ccppo sobre los que podemos acceder a información es de solo 11%. Asimismo, esta información correspondería únicamente a los ccpp urbanos



In [5]:
df1.describe()

,fre
count,3718.000000
mean,2100.066971
std,10528.450335
min,3.000000
25%,179.000000
50%,302.000000
75%,652.250000
max,314581.000000


In [63]:
Para estar seguro que los datos proporcionados por ambas fuentes de información son correctas unimos ambos DataFrame

11.77886900047521

In [62]:
print("Total de centros poblados con al menos 1 persona:",ccpp.query('pob_censada>50').count()[0])

Total de centros poblados con al menos 1 persona: 31565


In [26]:
m1=ccpp_5.merge(df1, right_index=True, left_index=True, how='outer', validate='1:1', indicator=True)
m1.groupby('_merge').size()

_merge
left_only     68381
right_only      115
both           3603
dtype: int64

In [51]:
print("Total de personas en ccpp sin ubigeo exacto:",m1.query('_merge=="right_only"')['Urbano censal'].sum(),"\n")
m1.query('_merge=="right_only"').head()

Total de personas en ccpp sin ubigeo exacto: 23953.0 



,codigo,ccpp_nombre,region_natural,altitud,pob_censada,hob_censados,muj_censados,viv_particulares,viv_ocupadas,viv_des,Urbano censal,_merge
ubigeo,,,,,,,,,,,,
0101058888,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,51.0,right_only
0205088888,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,right_only
0205098888,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,263.0,right_only
0215098888,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,88.0,right_only
0217078888,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,99.0,right_only


In [4]:
df=redapy.query_2017(tipo='Frequency', var1="vivienda.VAREA", area_break="centro poblado", selection="2301", censo='CPV2017_M')
#redapy.query_2017(tipo='Frequency', var1="vivienda.C2P13", area_break="distrito", selection="1501", print_query=True)
redapy.frequency(df, pivot=True)

Scrapeo iniciado
Se cargó página REDATAM con éxito
La tabla cargó completamente
Tabla scrapeada con éxito en: 0:00:09.185446


resp,Urbano censal
ubigeo,
2301010001,30558
2301010007,2908
2301020001,8636
2301020002,1121
2301030001,199
2301030011,422
2301040001,10079
2301040006,288
2301050001,388
